In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from PIL import Image

In [ ]:
df = pd.read_pickle("data/df_reduced.pkl")

In [ ]:
def get_distributions(df:pd.DataFrame, col:str):
    dc = df[col]
    dc.plot.hist(bins=90)
    plt.show()
    plt.close()
    print(dc.describe())
    return dc

In [ ]:
dc  = get_distributions(df, "cartesianZ")

In [ ]:
dc_hist = np.histogram(dc.values, bins=80)

In [ ]:
hist_weights = dc_hist[0] / dc_hist[0].sum()

In [ ]:
hist_centers = dc_hist[1]

In [ ]:
sampled_data = np.random.choice(hist_centers[1:], size=10000, p = hist_weights)

In [ ]:
gmm = GaussianMixture(5)
gmm = gmm.fit(sampled_data.reshape(-1,1))

In [ ]:
floor_id = np.argmin(gmm.means_)
ceil_id = np.argmax(gmm.means_)

floor_cutoff = (gmm.means_[floor_id] + np.sqrt(gmm.covariances_[floor_id])*2).reshape(-1)[0]
ceil_cutoff = (gmm.means_[ceil_id] - np.sqrt(gmm.covariances_[ceil_id])*2).reshape(-1)[0]

In [ ]:
floor_cutoff

In [ ]:
ceil_cutoff

In [ ]:
df = df[df.cartesianZ > floor_cutoff]
df = df[df.cartesianZ < ceil_cutoff-2]


In [ ]:
df_plot = df[["cartesianX", "cartesianY", "intensity"]]

In [ ]:
df_plot.plot.scatter("cartesianX", "cartesianY", s = 1, figsize=(15,10))
plt.show()

In [ ]:
# Quick experiment to pixel

In [ ]:
coords = df[["cartesianX", "cartesianY"]].copy()

minx = coords.cartesianX.min()
miny = coords.cartesianY.min()

coords.cartesianX -= minx
coords.cartesianY -= miny

coords.cartesianX *= 100
coords.cartesianX = coords.cartesianX.round().astype(np.int32)

coords.cartesianY *= 1000
coords.cartesianY = coords.cartesianY.round().astype(np.int32)

maxx = coords.cartesianX.max()
maxy = coords.cartesianY.max()

In [ ]:
matrix_bw = np.zeros([maxx+1, maxy+1])
matrix_bw[coords.cartesianX.values, coords.cartesianY.values] = 255

In [ ]:
img = Image.fromarray(matrix_bw, "L")
img.save('data/image.png')
img.show()